Copyright 2024 - Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Gemini Pro - Using LangChain to integrate with Vertex AI Search for RAG architecture.

* This notebook explains how to use grounding service in Gemini Pro.
* Refer to https://cloud.google.com/vertex-ai/generative-ai/docs/grounding/overview
* Using Vertex AI Search :
  * https://cloud.google.com/vertex-ai/generative-ai/docs/grounding/overview#ground-private
* Refer to the Langchain website.
  * https://python.langchain.com/docs/integrations/retrievers/google_vertex_ai_search

* Note:
  * /usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `GoogleVertexAISearchRetriever` was deprecated in LangChain 0.0.33 and will be removed in 0.3.0. An updated version of the class exists in the langchain-google-community package and should be used instead. To use it run `pip install -U langchain-google-community` and import as `from langchain_google_community import VertexAISearchRetriever`.


# Configuration
## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest


In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 5.2 MB/s eta 0:00:00


In [2]:
%pip install --upgrade --quiet google-cloud-discoveryengine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.1 MB/s eta 0:00:00


In [14]:
%pip install --upgrade --quiet langchain langchain_community

In [4]:
from IPython.display import display, Markdown

## Authentication to access to the GCP & Google drive

* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [5]:
#  For only colab to authenticate to get an access to the GCP.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

* Mount to the google drive to access the .ipynb files in the repository.

In [6]:
# To access contents in Google drive

if "google.colab" in sys.modules:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


# Execute the example
## Set the environment on GCP Project
* Configure project information
  * Model name : LLM model name : https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models
  * Project Id : prodect id in GCP
  * Region : region name in GCP

In [7]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [8]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part, Tool
import vertexai.generative_models as generative_models

# Grounding service is still in preview.
from vertexai.preview.generative_models import grounding

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)

# Access to the generative model.
model = GenerativeModel(MODEL_NAME)

### Configure Langchain function to integrate with the Vertex AI Search
* Currently using digital parser

In [17]:
from langchain.retrievers import GoogleVertexAIMultiTurnSearchRetriever
from langchain_google_community import VertexAISearchRetriever

# from langchain_community.retrievers import (
#     GoogleVertexAIMultiTurnSearchRetriever,
#     GoogleVertexAISearchRetriever,
# )
"""
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning:
The class `GoogleVertexAISearchRetriever` was deprecated in LangChain 0.0.33 and will be removed in 0.3.0.
An updated version of the class exists in the langchain-google-community package and should be used instead.
To use it run `pip install -U langchain-google-community` and import as `from langchain_google_community import VertexAISearchRetriever`.
"""

DS_REGION="global"
DATA_STORE_ID = "it-law-digital-parser_1718024672666"

retriever = GoogleVertexAISearchRetriever(
    project_id=PROJECT_ID,
    location_id=DS_REGION,
    data_store_id=DATA_STORE_ID,
    max_documents=3,
    # max_extractive_answer_count=2,
    # get_extractive_answers=True,
    max_extractive_segment_count=1,
    get_extractive_segments=True,
    query_expansion_condition=2,
    engine_data_type = 0
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `GoogleVertexAISearchRetriever` was deprecated in LangChain 0.0.33 and will be removed in 0.3.0. An updated version of the class exists in the langchain-google-community package and should be used instead. To use it run `pip install -U langchain-google-community` and import as `from langchain_google_community import VertexAISearchRetriever`.
  warn_deprecated(


### Query context from the Vertex AI Search

In [ ]:
query = "침해사고의 원인 분석에 대해서 설명해주세요."

result = retriever.get_relevant_documents(query)

context ={}

for doc in result:
    print(doc.json)
    context[doc.metadata['source']] = doc.page_content
    print("\n\n======================")
    print(doc.metadata['source'])
    print(doc.page_content)

print(context)



### Create model object with LangChain API

In [ ]:
from langchain_google_vertexai.llms import VertexAI

gemini_pro = VertexAI( model_name = MODEL_NAME,
                  project=PROJECT_ID,
                  location=REGION,
                  verbose=True,
                  streaming=False,
                  temperature = 0.2,
                  top_p = 1,
                  top_k = 40
                 )

### Request to the LLM

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate.from_template("""

  당신은 법률을 상담해주는 AI 어시스턴트입니다.
  아래 Question 에 대해서 반드시 Context에 있는 개별 내용을 기반으로 단계적으로 추론해서 근거를 설명하고 쉽게 답변해주세요.
  Context : {context}
  Question : {question}

  """)

prompt = prompt.format(context=context,
                       question=query)

print(f"Prompt : {prompt}")

response = gemini_pro.invoke(prompt)
display(Markdown(response))
